# Xavier's Sandbox Notebook

**Key Definitions:**
* FDR = Flow Direction Raster.
* FAC = Flow Accumulation Raster.
* Pour Point = The outlet point of a hydrologic unit/basin (i.e., FAC.max()).

But first run `conda-develop "C:\PATH\FCPGtools"` to get FCPGtools importable in the `/sandbox/` directory.

In [1]:
import FCPGtools as fc
import os
import rasterio as rs
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import subprocess

# Key Rasters

### ESRI NHDPlus Flow Direction Raster (FDR)
An example (from [`NHDPlusV21_MA_02_02a_FdrFac_01.7z`](https://www.epa.gov/waterdata/nhdplus-mid-atlantic-data-vector-processing-unit-02))of 8 bit flow direction raster (8 cardinal directions) using [NHDPlus](https://www.epa.gov/waterdata/get-nhdplus-national-hydrography-dataset-plus-data#v2datamap) raster coding convention (see below). 

**Cardinality:**
* 1 -> E (note that this is the only cardinal direction that does not need to be reclassified)
* 2 -> SE
* 4 -> S
* 8 -> SW
* 16 -> W
* 32 -> NW
* 64 -> N
* 128 -> NE

![Central PA FDR](imgs\esri_nhdplus_fdr_example.png)

# Key raster generation functions

## Reclassify ESRI flow direction raster (FDR) values into TauDEM FDR compatible values.

In [5]:
def tauDrainDir(inRast, outRast, band=1, updateDict={}):
    """
    Parameters
    inRast : str
        Path to a raster encoded with ESRI flow direction values.
    outRast : str
        Path to output a raster with flow directions encoded for TauDEM. File will be
        overwritten if it already exists.
    band : int (optional)
        Band to read the flow direction grid from if inRast is multiband, defaults to 1.
    updateDict : dict (optional)
        Dictionary of Rasterio raster options used to create outRast. Defaults have been supplied, 
        but may not work in all situations and input file formats.
    verbose : bool (optional)
        Print output, defaults to False.

    Returns
    -------
    outRast : raster
        Reclassified flow direction raster at the path specified above.
    """

    # assert that inRast is a file, if not return 'inRast not found'

    # use rasterio to open inRast, assert that the specified band exists

    # save a copy of the input metadata via inRast.profile.copy()

    # make a tauDir .copy() of the input raster and remap HNDplus flow directions to Taudem flow direction!
    # i.e., 2 -> 8, 4 -> 7, etc.

    # use updateDict to update the metadata

    # use rasterio to write tauDIR to the outRast .tif path

    return None

## Create a flow accumulation raster (FAC) from a TauDEM FDR
Wrapper for [TauDEM's AreaD8 (8 cardinal directions) accumulation area function](https://hydrology.usu.edu/taudem/taudem5/help53/D8ContributingArea.html). Currently the function returns nothing and simply writes to `accumRast` path using the TauDEM cmd line command. 

**Note:** The TauDEM function allows the sum of all upslope cells to be calculated, OR a weight can be added based on some other grid with the cmd line `[ -wg < wgfile >]` where `<wgfile>` is some parameter grid. Could we integrate this better?

In [9]:
def tauFlowAccum(fdr, accumRast, cores=1, mpiCall='mpiexec', mpiArg='-n', verbose=False) -> None:
    """Wrapper for TauDEM AreaD8 :cite:`TauDEM` to produce a flow accumulation grid.

    Parameters
    ----------
    fdr : str
        Path to a flow direction raster in TauDEM format.
    accumRast : str
        Path to output the flow accumulation raster.
    cores : int (optional)
        Number of cores to use. Defaults to 1.
    mpiCall : str (optional)
        The command to use for mpi, defaults to mpiexec.
    mpiArg : str (optional)
        Argument flag passed to mpiCall, which is followed by the cores parameter, defaults to '-n'.
    verbose : bool (optional)
        Print output, defaults to False.

    Returns
    -------
    accumRast : raster
        Raster of accumulated parameter values at the path specified above.
    """
    # format a dictionary with taudem inputs

    # construct the TauDEM AreaD8 command: '{mpiCall} {mpiArg} {cores} aread8 -p {fdr} -ad8 {outFl} -nc'.format(**tauParams)

    # use subprocesses to call the command. Write to accumRast.

    return None

## Create a parameter accumulation grid (in batch + one-by-one)
Basically uses TauDEM AreaD8.

**Base function notes:**
* Alters the original paramRast file, also generate
* Defaults to one core -> can we check cores and optimize? 
* Doesn't return anything, but does save the accumulated raster, nodata raster (nodata=1, data=0), and accumulated nodata raster. Note that during the workflow nodata values are converted to 0, **so the nodata accumulation raster could be used as an uncertainty accumulation model of sorts.**
* **Terrible behavior optionality for nodata handling!** Either you set all `param:outNoDataRast`, `param:outNoDataAccum`, and `param:zeroNoDataRast` to a valid path, or the nodata values are simply propagated. No option to not propogate but also not create intermediate rasters. Also no clear reason why a user wouldn't assume they could just set one of the paths.

**Batch version notes:** 
* Could be good to let it handle a batch using a directory input?
* Only behavior available is to automatically create and save nodata and nodata accumulation rasters. There really should be a way to avoid propagating nodata values without having to save all the intermediate products indescrimently. 

In [3]:
# notes provided in the docstring (in their words)
# """
# Notes
#    -----
#    If outNoDataRast, outNoDataAccum, and zeroNoDataRast inputs are all supplied and there are "no data" values in the
#    basin this function will set any “no data” values in the basin to zero and save that raster as zeroNoDataRast.
#    This function will then save a raster with all no data values set to one and other values set to zero
#    (outNoDataRast) and use tauDEM to accumulate it (outNoDataAccum). It will then accumulate the parameter from the
#    zeroNoDataRast, and a subsequent correction will be needed in the make_fcpg() function based on the values
#    in the outNoDataAccum raster.
#
#    If some of the output file locations for handling no data values aren’t supplied or “no data” values aren’t present
#    in the parameter grid, it will simply accumulate the parameter grid. If “no data” values are present, this will
#    result in them being propagated downstream.
# """

In [ ]:
def accumulateParam(paramRast, fdr, accumRast, outNoDataRast=None, outNoDataAccum=None, zeroNoDataRast=None, cores=1,
                    mpiCall='mpiexec', mpiArg='-n', verbose=False) -> None:
    """Accumulate a parameter grid using TauDEM AreaD8 :cite:`TauDEM`.

    Parameters
    ----------
    paramRast : str
        Raster of parameter values to accumulate; this file is modified by the function.
    fdr : str
        Flow direction raster in TauDEM format.
    accumRast : str
        File location to store accumulated parameter values.
    outNoDataRast : str (optional)
        File location to store parameter no data raster.
    outNoDataAccum : str (optional)
        File location to store accumulated no data raster.
    zeroNoDataRast : str (optional)
        File location to store the no data raster filled with zeros.
    cores : int (optional)
        The number of cores to use for parameter accumulation. Defaults to 1.
    mpiCall : str (optional)
        The command to use for mpi, defaults to mpiexec.
    mpiArg : str (optional)
        Argument flag passed to mpiCall, which is followed by the cores parameter, defaults to '-n'.
    verbose : bool (optional)
        Print output, defaults to False.

    Returns
    -------
    accumRast : raster
        Raster of accumulated parameter values.
    outNoDataRast : raster
        Raster of no data values.
    outNoDataRast : raster
        Raster of accumulated no data values.
    """
    # open the parameter raster and copy it's metadata as well a it's nodata value

    # (optional) open the FDR and copy it's nodata value

    # (optional) check the # of cells that are nodata on the parameter grid, but not nodata on the FDR
    # (optional) if such cells are present, create a copy of the parameter grid and set the cells to 0
    # (optional) write metadata/rs.profile for the copied raster and save it.

    # (optional) create a second copy of the parameter grid, where nodata values (that are not nodata on the FDR) = 1,
    # (optional) and data = 0, and out of FDR bounds (aka FDR = nodata) = -1.
    # (optional) Convert to a 8-bit array, then write new metadata settin nodata = -1, then save as a GeoTIFF. 

    # (optional) use tauDEM AreaD8 to accumulate nodata values

    # create accumulation grid using either the nodata->0 or raw parameter grid

    return None


def accumulateParam_batch(paramRasts, fdr, outWorkspace, cores=1, appStr="accum", mpiCall='mpiexec', mpiArg='-n',
                          verbose=False):
    """Batch version of :py:func:`accumulateParam`.

    Parameters
    ----------
    paramRasts : list
        List of input parameter raster paths to accumulate along the supplied fdr.
    fdr : str
        Path to the flow direction raster.
    outWorkspace : str
        Path to the output directory for accumulation rasters.
    cores : int (optional)
        Number of cores to use. Defaults to 1.
    appStr :str (optional)
        String of text to append to accumulated parameter filenames. Defaults to "accum."
    mpiCall : str (optional)
        MPI program to use to execute the program, defaults to mpiexec.
    mpiArg : str (optional)
        Argument to pass to mpiCall, defaults to '-n'.
    verbose : bool (optional)
        Print output, defaults to False.

    Returns
    -------
    fileList : list
        List of file paths to accumulated parameter rasters.
    """
    # iterate thru each paramRast path

    # create the nodata rasters (only behavior available) by creating outfile names

## Use flow accumulation boundary conditions via upstream pour points to update FAC raster
**Notes:**
* You can generate an updated FAC using an existing FAC or FDR?

In [ ]:
def adjustFAC(facWeighttemplate, downstreamFACweightFl, updateDictFl, downstreamFDRFl, adjFACFl, cores=1,
              mpiCall='mpiexec', mpiArg='-n', verbose=False, scaleFactor=None, moveDownstream=False):
    """Generate an updated flow accumulation grid (FAC) given an update dictionary produced by :py:func:`createUpdateDict`.

    Parameters
    ----------
    facWeighttemplate : str
        Path to a FDR or FAC grid used to make the FAC weighting grid.
    downstreamFACweightFl : str
        Path to output the FAC weighting grid.
    updateDictFl : str
        Path to update dictionary used to update the FAC weighting grid.
    downstreamFDRFl : str
        Path to downstream FDR to use when computing the adjusted FAC grid.
    adjFACFl : str
        Path to output the adjusted FAC raster.
    cores : int (Optional)
        Number of cores to use. Defaults to 1.
    mpiCall : str (optional)
        MPI program to use to execute the program, defaults to mpiexec.
    mpiArg : str (optional)
        Argument to pass to mpiCall, defaults to -n.
    verbose : bool (optional)
        Print output, defaults to False.
    scaleFactor : int (optional)
        Value to divide weighting grid by for working with very large flow accumulation values and
        associated parameter value, defaults to None.
    moveDownstream : bool (optional)
        Move the pour point downstream to account for pour points that do not overlap downstream
        geospatial tiles, defaults to False.

    Returns
    -------
    adjFACFl : raster
        Adjusted flow accumulation raster at adjFACFl.
    """

# Make a Flow Conditioned Parameter Grid (FCPG) raster
* The FCPG is essentially just the parameter accumulation raster divided by the FAC raster. Therefore higher FCPG values indicate cells where parameter accumulation is high relative to cell accumulation/upstream area, while lower values indicate low parameter accumulation relative to the upstream area. **Note that these values are non-normalized, so they are only intepretable relative to the units of the parameter grid!** 
* No geoprocessing (TauDEM or GDAL) required, however the inputs are the results of geoprocessing.

In [1]:
def make_fcpg(accumParam, fac, outRast, noDataRast=None, minAccum=None,
              ESRIFAC=False, verbose=False) -> None:
    """Create a flow-conditioned parameter grid using accumulated parameter and area rasters.

    Parameters
    ----------
    accumParam : str
        File location of the accumulated parameter data raster.
    fac : str
        File location of the flow accumulation raster.
    outRast : str
        File location of the output flow-conditioned parameter grid.
    noDataRast : str (optional)
        File location of the accumulated parameter no data raster.
    minAccum : float (optional)
        Value of flow accumulation below which the CPG values will be set to no data.
    ESRIFAC : bool (optional)
        Use an ESRI FAC grid, defaults to False. ESRI-derived FAC grids have zeros for the first
        cell in a flowpath, as such, a 1 is added to the grid.
    verbose : bool (optional)
        Print output, defaults to False.

    Returns
    -------
    outRast : raster
        Flow-conditioned parameter grid file where grid cell values represent the mean upstream
        value of the paramter.
    """

    # open the accumulated parameter data raster

    # convert to float32 dtype and convert nodata to np.NaN

    # open the flow accumulation raster (add +1 to every cell is ESRIFAC=True)

    # if a nodata raster was provided convert nodata cells to np.Nan
    # this removes cells from the FAC that are nodata in the parameter grids

    # check for eroneous negative accumulationvalues (no automatic handling of them)

    # create dataCPG = paramAccum / FAC (both float64 with np.NaN)

    # create updated metadata and save the dataCPG

    return None


def make_fcpg_batch(accumParams, fac, outWorkspace, minAccum=None, appStr="FCPG",
                    ESRIFAC=False, verbose=False) -> list:
    """Batch version of :py:func:`make_fcpg`.
    Parameters
    ----------
    accumParams : list
        List of accumulated parameter rasters to create FCPGs from.
    fac : str
        Path to the flow accumulation raster.
    outWorkspace : str
        Path to an output directory for produced FCPGs.
    minAccum : int (optional)
        Minimum accumulation value below which the output FCPG will be turned to no data values.
        Defaults to None.
    appStr : str (optional)
        String of text to append to filenames of the produced FCPG grids.
    ESRIFAC : bool (optional)
        Use an ESRI FAC grid, defaults to False. ESRI-derived FAC grids have zeros for the first cell
        in a flowpath, as such, a 1 is added to the grid.
    verbose : bool (optional)
        Print output, defaults to False.

    Returns
    -------
    fileList : list
        List of file paths to the produced FCPGs.
    """
    return list

# "Non-core" raster functionality
The following functions either rely on, or directly feed into, TauDEM functions that are not [D8 Contributing Area](https://hydrology.usu.edu/taudem/taudem5/help53/D8ContributingArea.html).

## Make streams - `makeStreams()` -> maybe include in core?
* The idea is essentially it returns a raster with cell value = 1 if above some FAC accumulation threshold, with other cells masked out.

In [ ]:
def makeStreams(fac, strPath, thresh=900, updateDict=dict:
    """
    Create stream grid from a flow accumulation grid based on a threshold value.

    Parameters
    ----------
    fac : str
        Path to the flow accumulation grid that will be used to create the stream grid.
    strPath : str
        Path to output the stream grid where stream cells will be 1 and other cells will be the no-data value from the source fac grid.
    thresh : int (optional)
        Flow accumulation threshold above which streams are created, defaults to 900 cells.
    updateDict : dict (optional)
        Rasterio raster creation parameters.
    verbose : bool (optional)
        Print output, defaults to False.

    Returns
    -------
    None

    """
    # open the fac raster
    # create a numpy grid of zeros (matching FAC dimensions) and convert to all nan

    # use indexing to replace all values in empty raster with 1 if the equal index cell in the FAC is >= param:thresh
    return None

## Calculate cell distances from stream - `distance_to_stream()`
* Uses TauDEM's [D8 Distance to Streams](https://hydrology.usu.edu/taudem/taudem5/help53/D8DistanceToStreams.html) function to find distance to nearest value=1 cell in a stream mask raster (as produced by `makeStreams()`).
* **Note:** The existing function used an FAC raster as the `srcfile` input into the TauDEM cmd line call. **This is a wierd way to do it!** The `srcfile` could just be the stream mask raster, and then the default of 1 can be used, however here we have a stream mask function, as well as this function that applies the exact same threshold for the distance calculation. Idk I guess it's OK?

In [ ]:
def distance_to_stream(fdr, fac, thresh, outRast, cores=1, mpiCall='mpiexec', mpiArg='-n', verbose=False):
    """
    Compute distance to streams.

    Parameters
    ----------
    fdr : str
        Path to flow direction raster in TauDEM format.
    fac : str
        Path to flow accumulation raster.
    thresh : int
        Accumulation threshold for stream formation in number of grid cells.
    outRast : str
        Path to output the distance raster.
    cores : int (optional)
        The number of cores to use. Defaults to 1.
    mpiCall : str (optional)
        The command to use for mpi, defaults to mpiexec.
    mpiArg : str (optional)
        Argument flag passed to mpiCall, which is followed by the cores parameter, defaults to '-n'.
    verbose : bool (optional)
        Print output, defaults to False.

    Returns
    -------
    outRast : raster
        Raster with values of D-8 flow distance from each cell to the nearest stream.
    """
    # use taudem d8hdisttostrm cmd line call
    pass

## Make a Decay Grid based on the inverse of the distance to stream - `makeDecayGrid()`
* `decayGrid = np.exp((-1 * decayGrid * xsize) / (xsize ** k))` where `decayGrid` is a distrance to stream raster, `xisze` is read from the input raster, and `k` is a constant that controls the decay rate. >2 is slower, <2 is faster, 2=moderate.
* **No TauDEM** dependency directly.

In [ ]:
def makeDecayGrid(d2strm, k, outRast, verbose=False):
    """
    Create a decay raster where grid cell values are computed as the inverse number of grid cells,
    :code:`np.exp((-1 * decayGrid * xsize) / (xsize ** k))`, where decayGrid is the distance from the d2strm
    raster from each grid cell to the nearest stream, k is a constant applied to the cell size values,
    and dx is the cell size of the raster, pulled from the d2strm raster directly.

    Parameters
    ----------
    d2strm : str
        Path to raster of flow distances from each grid cell to the nearest stream.
    k : float
        Dimensionless constant applied to decay factor denominator.
        Set k to 2 for "moderate" decay; greater than 2 for slower decay; or less than 2 for faster decay.
    outRast : str
        Output file path for decay grid.
    verbose : bool (optional)
        Print output, defaults to False.

    Returns
    -------
        outRast : raster
            Raster file with grid cells values representing weights decaying as a function of the distance to stream.

    Notes
    -----
    The decay equation in this tool has changed between version 1.0.2 and version 1.0.3.
    The new equation is :code:`np.exp((-1 * decayGrid * xsize) / (xsize ** k))`.
    The original equation was :code:`decayGrid = xsize/(decayGrid + k*xsize)`.
    """
    # read in param:d2strm and get it's cell size
    # change the dtype of param:d2strm to float32
    # evaluate np.exp((-1 * decayGrid * xsize)/(xsize ** k))
    # replace np.nan values with some nodata value
    pass

## Make a decay accumulation raster from a D-Infinity FDR - `decayAccum()`
* Wrapper of TauDEM's [D-Infinity Decaying Accumulation
](https://hydrology.usu.edu/taudem/taudem5/help53/DInfinityDecayingAccumulation.html) function.
* From TauDEM docs: "The D-Infinity Decaying Accumulation tool **creates a grid of the accumulated quantity at each location in the domain where the quantity accumulates with the D-infinity flow field, but is subject to first order decay in moving from cell to cell.** By default, the quantity contribution of each grid cell is the cell length to give a per unit width accumulation, but can optionally be expressed with a weight grid. The decay multiplier grid gives the fractional (first order) reduction in quantity in accumulating from grid cell x to the next downslope cell."

In [ ]:
def decayAccum(ang, mult, outRast, paramRast=None, cores=1, mpiCall='mpiexec', mpiArg='-n', verbose=False):
    """
    Decay the accumulation of a parameter raster.

    Parameters
    ----------
    ang : str
        Path to flow angle raster from the TauDEM D-Infinity flow direction tool.
    mult : str
        Path to raster of multiplier values applied to upstream accumulations,
        1 corresponds to no decay, 0 corresponds to complete decay.
    outRast : str
        Path to output raster for decayed accumulation raster.
    paramRast : str (optional)
        Raster of parameter values to accumulate. If not supplied area will be accumulated. Defaults to None.
    cores : int (optional)
        Number of cores to use. Defaults to 1.
    mpiCall : str (optional)
        The command to use for mpi, defaults to mpiexec.
    mpiArg : str (optional)
        Argument flag passed to mpiCall, which is followed by the cores parameter, defaults to '-n'.
    verbose : bool (optional)
        Print output, defaults to False.

    Returns
    -------
    outRast : raster
        Decayed accumulation raster, either area or parameter depending on what is supplied to the function.
    """
    # make TauDEM:dinfdecayaccum cmd line call
    # if param:paramRast is not None, the parameter raster is used to accumulate

## Wrap TauDEM's [D8 Extreme Upslope Value](https://hydrology.usu.edu/taudem/taudem5/help53/D8ExtremeUpslopeValue.html) function - `ExtremeUpslopeValue()`
* Finds the furthest upstream point elevation (or other) value from a given point on a D8 Flow Direction Raster (FDR).
* `param:accum_type` controls whether the max upslope value, or min upslope value is returned.

In [ ]:
def ExtremeUpslopeValue(fdr, param, output, accum_type="MAX", cores=1, fac=None, thresh=None, mpiCall='mpiexec',
                        mpiArg='-n', verbose=False):
    """
    Wrapper for the TauDEM D8 Extreme Upslope Value function :cite:`TauDEM`.

    Parameters
    ----------
    fdr : str
        Path to a flow direction grid in TauDEM format.
    param : str
        Path to parameter raster to run through the D8 Extreme Upslope Value tool
    output : str
        Path to output raster file.
    accum_type : str (optional) 
        Either  "MAX" or "MIN." Defaults to "MAX."
    cores : int (optional) 
        Number of cores to run this process on. Defaults to 1.
    fac : str (optional)
        Path to a flow accumulation raster. Defaults to None.
    thresh : int (optional)
        Threshold values, in the same units as fac to mask output to stream channels. Defaults to None.
    mpiCall : str (optional)
        The command to use for mpi, defaults to mpiexec.
    mpiArg : str (optional)
        Argument flag passed to mpiCall, which is followed by the cores parameter, defaults to '-n'.
    verbose : bool (optional)
        Print output, defaults to False.

    Returns
    -------
    output : raster
        Raster of either the maximum or minumum upslope value of the parameter grid supplied to the function.
    """
    # use tauDEM:d8flowpathextremeup

    # apply a threshold (optional) where the low accumulation cells become nodata
    # if param:thresh and param:fac are not None. Uses numpy indexing.
    return None

# Parameter grid prep functions

## Resample + reproject + clip a raster to match a FDR raster
* Uses **GDALWarp** to prepare an arbitrary raster to match cell size, projection, and extent.
* **Question:** Do the cells align? Is that a part of GDALWarp (i.e., simular to [ArcPro Snap Raster](https://desktop.arcgis.com/en/arcmap/10.3/tools/environments/snap-raster.htm)).

**GDAL Warp Parameters:**
```
{'inParam': inParam,
                'outParam': outParam,
                'fdr': fdr,
                'cores': str(cores),
                'resampleMethod': resampleMethod,
                'xsize': xsize,
                'ysize': ysize,
                'fdrXmin': fdrXmin,
                'fdrXmax': fdrXmax,
                'fdrYmin': fdrYmin,
                'fdrYmax': fdrYmax,
                'fdrcrs': fdrcrs,
                'nodata': paramNoData,
                'datatype': outType}
```
**Note:** Currently cores defaults to 1, this requires users to know how many cores their computer has. It could be cool to check how many cores the computer has and use all of them if a boolean parameter is true (i.e., `optimize_cores:bool = True`).

In [6]:
def resampleParam(inParam, fdr, outParam, resampleMethod="bilinear", cores=1, forceProj=False,
                  forceProj4="\"+proj=aea +lat_1=29.5 +lat_2=45.5 +lat_0=23 +lon_0=-96 +x_0=0 +y_0=0 +ellps=GRS80 +datum=NAD83 +units=m +no_defs\"",
                  verbose=False):
    """
    Parameters
    ----------
    inParam : str
        Path to the input parameter data raster
    fdr : str
        Path to the flow direction raster
    outParam : str
        Path to the output file for the resampled parameter raster.
    resampleMethod : str (optional)
        resampling method, either 'bilinear' or 'near' for nearest neighbor. Bilinear should 
        generally be used for continuous data sets such as precipitation while nearest neighbor
        should generally be used for categorical 
        datasets such as land cover type. Defaults to bilinear.
    cores : int (optional)
        The number of cores to use. Defaults to 1.
    forceProj : bool (optional)
        Force the projection of the flow direction raster. This can be useful if the flow
        direction raster hasan unusual projection. Defaults to False. This parameter defaults
        to False; however, if set to True, forceProj4 must also be specified or the default
        proj4 string for USGS Albers will be used, see below.
    forceProj4 : str (optional)
        Proj4 string used to force the flow direction raster. This defaults to USGS Albers, but is not used 
        unless the forceProj parameter is set to True.
    verbose : bool (optional)
        Print output, defaults to False.

    Returns
    -------
    outParam : raster
        Resampled, reprojected, and clipped parameter raster.
    """
    # open in FDR raster with rasterop amd ge the coorindate system, cell size, and bounding coordinates

    # open the input raster and get the same attributes as the FDR raster (+ first band dtype)

    # get the output dtype. If it's 8-but, convert to 16but using a possibly outdated GDAL  `outType = 'Byte'`

    # check if resampling or repojection is required, if not, copy the metadata + raster as is

    # if things need to be changeg use a `gdalwarp` command line cmd via `subprocess.run(cmd, shell=True)`
    warpParams = {'dict of': 'gdal warp parameters'}

    cmd = 'gdalwarp -overwrite -tr {xsize} {ysize} -t_srs {fdrcrs} -te {fdrXmin} {fdrYmin} {fdrXmax} {fdrYmax} \
    -co "PROFILE=GeoTIFF" -co "TILED=YES" -co "SPARSE_OK=TRUE" -co "COMPRESS=LZW" -co "ZLEVEL=9" -co "NUM_THREADS={cores}" \
     -co "BIGTIFF=IF_SAFER" -r {resampleMethod} -dstnodata {nodata} -ot {datatype} {inParam} {outParam}'.format(
                **warpParams)

    result = subprocess.run(cmd, shell=True)
    result.stdout
    # note that the result.stdout output is calculated but not returned nor saved to a variable
    return None

## Convert a categorical raster into a set of binary rasters (raster "one-hot-encoding")
* All rasters are unique, this is a great application for multi-band xarray rasters allowing one to be saved and limiting possibility of altering one file detrimentally. This would be especially relevant with more categories.

As it sounds. A categorical raster (i.e., land cover) with N unique values is converted into N binary rasters where the value is either 1, 0, or -1 for nodata. **Output raster paths are returned as a list**. 

**Explanation:** The idea here is to allow the accumulation of categorical cells by coding each class as 1 in it's own 8bit raster (-1-nodata).

**Ideas:**
* Is a list really the best data structure to return binary rasters? Could I dictionary be better? 
* Could weighting be useful? Why do all categories have to accumulate the same? (or is this better controlled in the accumulation function?)
* Could saving each category as a band in a multi-band raster (with updated metadata defining classes) have any benefit?

Main function is `cat2bin(inCat, outWorkspace, par=True, verbose=False)`, which just parallelizes `binarizeCat(...)` via multiprocessing. 

In [8]:
def binarizeCat(val, data, nodata, outWorkspace, baseName, ext, profile, verbose=False):
    """
    Turn a categorical raster (e.g. land cover type) into A SINGLE BINARY RASTER PER UNIQUE CATEGORY/val, 
    0 for areas where that class is not present, 1 for areas where that class is present, and -1 for
    regions of no data in the supplied raster. Used in :py:func:`cat2bin`.

    Parameters
    ----------
    data : np.array
        Numpy array of raster data to convert to binary.
    val : int
        Raster value to extract binary for from data.
    nodata : int or float
        Raster no data value.
    outWorkspace : str
        Path to folder to save binary output rasters to.
    baseName : str
        Base name for the output rasters.
    ext : str
        File extension for output rasters.
    profile : dict
        Rasterio metadata dictionary decribing the properties used to create the output raster.
    verbose : bool (optional)
        Print output, defaults to False.

    Returns
    -------
    catRaster : str
        Filepath to the binary raster created."""
    # catData = data.copy()
    # catData[(data != val) & (data != nodata)] = 0
    # catData[data == val] = 1
    # catData[data == nodata] = -1  # Use -1 as no data value
    # catData = catData.astype('int8')

    # writes raster and returns it's path
    return str


def cat2bin(inCat, outWorkspace, par=True, verbose=False) -> list:
    """"
    TLDR:Basically applies binarizeCat() in parallel and handles metadata writing.

    Description - 
    Turn a categorical raster (e.g. land cover type) into a set of binary rasters, one for each category in the
    supplied raster, zero for areas where that class is not present, 1 for areas where that class is present,
    and -1 for regions of no data in the supplied raster. Wrapper on :py:func:`binarizeCat`.

    Parameters
    ----------
    inCat : str
        Input categorical parameter raster.
    outWorkspace : str
        Workspace to save binary raster output files.
    par : bool (optional)
        Use parallel processing to generate binary rasters, defaults to True.
    verbose : bool (optional)
        Print output, defaults to False.

    Returns
    -------
    fileList : list
        List of filepaths to output files."""

    from functools import partial
    # Open raster with rasterio and copy metadata

    # use numpy to find unique raster values and drop nodata categories

    # use multiprocessing processPool for multi-threaded raster processing (if par=True)

    # use the binarizeCat() function to produce a list of binary rasters
    # fileList = pool.map(partial(binarizeCat, data=dat, nodata=nodata,
    # outWorkspace=outWorkspace, baseName=baseName, ext=ext, profile=profile), cats)

    # basically pool.map(partial(binarizeCat), cats) parallelizes val=cat for call N cats.
    return list

# Shapefile functions (pour points and basins)

## Add columns to a HUC12 level basin geoDF indicating their HUC4 level basin membership (`HUC4`), and the HUC4 basin they pour into `toHUC4`.


In [ ]:
def makePourBasins(wbd, fromHUC4, toHUC4, HUC12Key='HUC12', ToHUCKey='ToHUC') -> gpd.GeoDataFrame:
    """Make geodataframe of HUC12 basins flowing from HUC4 to toHUC4.

    Parameters
    ----------
    wbd : GeoDataframe
        HUC12-level geodataframe projected to the same coordinate reference system (CRS) as the flow accumulation (FAC)
        and flow direction (FDR) grids being used.
    fromHUC4 : str
        HUC4 string for the upstream basin (i.e., '1407').
    toHUC4 : str
        HUC string for the downstream basin (i.e., '1501').
    HUC12Key : str (optional)
        Column name for HUC codes to process down to HUC4 codes, defaults to 'HUC12'.
    ToHUCKey : str (optional)
        Column name for the column that indicates the downstream HUC for each row of the dataframe, defaults to 'TOHUC'.

    Returns
    -------
    pourBasins : GeoDataframe
        HUC12-level geodataframe of units that drain from fromHUC4 to toHUC4.
    """
    # NOTE: getHUC4() is literally just in:str -> out:str[:4] which pulls HUC12 ID from the wbd['HUC12'] column

    # wbd['HUC4'] = wbd[HUC12Key].map(getHUC4) - gets the HUC12 basin's HUC4 level basin membership
    # wbd['ToHUC4'] = wbd[ToHUCKey].map(getHUC4) - gets the output HUC12 basin's (i.e., 'toHUC') HUC4 level equivalency

    # returns a copy of geodataframe rows where HUC4 = param:fromHUC4 and toHUC4 = param:toHUC4
    return gpd.GeoDataFrame

## Find/store pour points for each HUC4 basin using the basin geoDF, FAC, and FDR. 
**Note:** Pour points are the **outlet** of a basin, defined as having the maximum Flow Accumulation Raster value. 

There are two key functions here:
* `findPourPoints(basins, FAC, FDR)` - geospatial analysis to find outlet points in the pour basins (i.e., where fromHUC4 and toHUC4 match what is desired).
* `createUpdateDict(x: list, y: list, upstreamFACmax: list, fromHUC: str, out_json_path): str` - which creates a summary dictionary of pour points.
   *  Example output: ```{'1407': {'x': ['-1370669.9999999995', '-1371809.9999999995'],
   'y': ['1648259.9999999963', '1647779.9999999963'],
  'maxUpstreamFAC': ['155007.0', '101615.0'],
  'vars': ['maxUpstreamFAC']}}```
  
**Explanation:** Basically this finds the maximum flow accumulation cell for each HUC12 basin. This cell's location represents the pour point of the basin, and would act as a boundary condition for modelling the downstream HUC4 OR HUC12 level basin.  **<-----IMPORTANT**

**Ideas:** 
* Theoretically you could then have HUC4 pour into a larger basin? Is there utility in this? Could the pour points and basin level hierarchy be generalized to work at any basin level? Or maybe not generalized but added as a feature?
* There is an easy 2.8ms performance loss everytime a raster is re-opened from path rather than just passing the object (tested via %%timeit). Fpor example, for every pour point queryPoint() is used, which re-opens a raster that could just be opened once. Maybe `raster:Union['str'/PATH, rs.Raster]`?

In [1]:
def findPourPoints(pourBasins, upfacfl, upfdrfl, plotBasins=False):
    """Finds unique pour points between two HUC4s.

    Parameters
    ----------
    pourBasins : GeoDataframe
        GeoDataframe of the HUC12 basins that flow into the downstream HUC4. Used to clip the upstream FAC grid to
         identify pour points.
    upfacfl : str
        Path to the upstream flow accumulation grid.
    upfdrfl : str
        Path to the upstream tauDEM flow direction grid.
    plotBasins : bool (Optional)
        Boolean to make plots of upstream HUC12s and identified pour points. Defaults to False.

    Returns
    -------
    finalPoints : list
        List of tuples containing (x,y,w). These pour points have not been incremented downstream and can be used to
         query accumulated (but not FCPGed) upstream parameter grids for information to cascade down to the next
          hydrologic region / geospatial tile downstream.
    """
    # make an empty list to store pour points pourPoints = []

    # iterate over each HUC12 basin in the geoDF that flows into the downstream HUC4 basin. --------------
    # get the HUC boudnary and make a shape out of it using getFeatures() -> list of coordinates

    # apply a mask on the FAC raster using the HUC boundary coorindates -> np.array and affine-transform

    # find the maximum flow accumulation value points array index (could be multiple, non georeferences)

    # get coorindates of the max FAC points by trnasforming their np index via the affine-transforn

    # zip the coorindates of al max elevation points / pour
    # ------------------------------

    # with the list of xy pour points, verify that their downstream cell (via FindDownstreamCellTauDir()) IS Nodata!
    # Note: this makes sure the pour point (i.e., max accumulation cell) is at the edge of the basin and not some pit.

    # if no pour point is found with a NoData cell adjacent -> just append the pout point as is? (seems poorly handled...)
    # this is applied using queryPoints(), which needlessly uses a loop to get the value from the raster (as well as re-opening it from path)

    # get unique pour points and return in a list of tuples finalPoints = [(x, y, w), ...]

    return list  # of (x, y, w) points where w = FAC value


def createUpdateDict(x, y, upstreamFACmax, fromHUC, outfl, replaceDict=True, verbose=False, outletX=None, outletY=None) -> dict:
    """Create a dictionary for updating downstream FAC and parameter grids using values pulled from the next grid upstream.

    Parameters
    ----------
    x : list
        Horizontal coordinate(s) for where the update needs to happen in the downstream grid.
    y : list
        Vertical coordinate(s) for where the update needs to happen in the downstream grid.
    upstreamFACmax : list
        Value(s) to insert into the downstream FAC grid (i.e., w values from findPourPoints()).
    fromHUC : str
        The upstream HUC that the values are coming from.
    outfl : str (path)
        Path to where to save the json of this dictionary. The convention is to name this by the downstream HUC.
    replaceDict : bool (optional)
        Replace the update dictionary instead of updating with a new value. Defaults to True.
    verbose : bool (optional)
        Print output, defaults to False.
    outletX : list (optional)
        Outlet horizontal coordinate(s) if different from x above, defaults to None.
    outletY : list (optional)
        Outlet vertical coordinate(s) if different from y above, defaults to None.

    Returns
    -------
    updateDict : dict
        Update dictionary that is also written to outfl.
    """

    # { 'fromHUC': {  # make dictionary for the upstream FAC
    #    'x': xs,
    #    'y': ys,
    #    'maxUpstreamFAC': facs,
    #    'vars': ['maxUpstreamFAC']}}
    return dict

## Get the coorindates and max value value from an arbitrary upstream raster (commonly raw cell accumulation) - strange implementation?
**Notes:** 
* Finds a **single raster pourpoint.**
* Current behavior requires a FAC grid regardless of whether an optional accumulated parameter grid is provided. **This should be generalized to just get the pour point accumulation value of either a raw cell or parameter accumulation grid.**
* This function just gets the max accumulation value and info about it (coorindates + cell size)! **This is distinct from getting the pour point values because pour points are basin dependent!** Therefore FAC raster that has not been pre-masked by a basin will not return multiple basin pour point values. **<--------- IMPORTANT**
* The column/row index of the max value is converted to CRS coordinates using the projection of the FAC dataset. This means that if the FAC and parameter grid raster's are unaligned (either projection or coverage), then the coordinates would be false as it is merely a conversion of relative positioning on the input FAC (`param:facfl`) raster. **This only makes sense with the functions default behavior (i.e., `:param:fl==None`).**

In [ ]:
def findLastFACFD(facfl, fl=None) -> list:
    """Find the coordinate of the greatest cell in facfl, return the value from fl at that point.

    Parameters
    ----------
    facfl : str
        Path to a flow accumulation grid.
    fl : str (optional)
        Path to an accumulated parameter file. Defaults to None. If None, the facfl is queried.

    Returns
    -------
    x : float
        Horizontal coordinate of the greatest FAC cell.
    y : float
        Vertical coordinate of the greatest FAC cell.
    d : float
        Value from the parameter grid queried.
    w : float
        Cell size of the grid.


    Notes
    -----
    This can be used to find the flow direction of the FAC cell with the greatest accumulation value or the parameter value of the cell with the greatest accumulation value.
    """
    # load the FAC raster (regardless of whether it get's used...)

    # find the column/row index of the max value using numpy.max()

    # query the cell value using the column/row index (seems redundant since we already got the value using numpy.max())

    # ASSUMING THE FAC RASTER AND THE FL RASTER ARE ALIGNED (UNCHECKED), use the column/row index and the existing
    # CRS projection in the FAC raster (param:facfl) to get the assumed coorindates of the max value. 

    # get the cell size from the metadata

    return list  # x, y, d, w